In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from os.path import join
from collections import defaultdict
import matplotlib.pyplot as plt
import brainbox.io.one as bbone
import sys 
sys.path.append('..')
from reproducible_ephys_functions import query
from tqdm import notebook
import brainbox as bb
from ibllib.io import spikeglx
import brainbox.behavior.wheel as wh
from brainbox import singlecell
from brainbox.metrics.single_units import spike_sorting_metrics
from one.api import ONE
import alf

from ibllib.qc.camera import CameraQC
import os
from scipy.stats import zscore
from collections import Counter

from utils import *

one = ONE()

In [ ]:
# mtnn_criteria = check_mtnn_criteria()

In [ ]:
# # load all rs trajectories
# trajs = query()

In [ ]:
# for eid in mtnn_criteria.keys():
#     print(eid)
#     print(mtnn_criteria[eid])
#     for traj in trajs:
#         if traj['session']['id'] == eid:
#             print(traj)
#             print(traj['session']['start_time'].split('T')[0]+'-'+str(traj['session']['number']))
#     print('------------------------------------')

In [ ]:
# for eid in get_mtnn_eids().keys():
#     print(eid)

#     stimOn_times = one.load_dataset(eid, '_ibl_trials.stimOn_times.npy')
#     firstMovement_times = one.load_dataset(eid, '_ibl_trials.firstMovement_times.npy')
#     #print(np.nanmin(np.diff(firstMovement_times, prepend=0)))

#     goCue_times = one.load_dataset(eid, '_ibl_trials.goCue_times.npy')

#     choice = one.load_dataset(eid, '_ibl_trials.choice.npy')
#     response_times = one.load_dataset(eid, '_ibl_trials.response_times.npy')

#     feedbackType = one.load_dataset(eid, '_ibl_trials.feedbackType.npy')
#     feedback_times = one.load_dataset(eid, '_ibl_trials.feedback_times.npy')

#     stimOff_times = one.load_dataset(eid, '_ibl_trials.stimOff_times.npy')
    
#     stimOff_diff = np.diff(stimOff_times, prepend=0)
    
#     choice_filter = np.where(choice!=0)

#     stimOn_times = stimOn_times[choice_filter]
#     firstMovement_times = firstMovement_times[choice_filter]
#     goCue_times = goCue_times[choice_filter]
#     response_times = response_times[choice_filter]
#     choice = choice[choice_filter]
#     feedbackType = feedbackType[choice_filter]
#     feedback_times = feedback_times[choice_filter]
#     stimOff_times = stimOff_times[choice_filter]
#     stimOff_diff = stimOff_diff[choice_filter]

#     nan_idx = set()
#     nan_idx.update(np.where(np.isnan(stimOn_times))[0].tolist())
#     nan_idx.update(np.where(np.isnan(firstMovement_times))[0].tolist())
#     nan_idx.update(np.where(np.isnan(goCue_times))[0].tolist())
#     nan_idx.update(np.where(np.isnan(response_times))[0].tolist())
#     nan_idx.update(np.where(np.isnan(feedback_times))[0].tolist())
#     nan_idx.update(np.where(np.isnan(stimOff_times))[0].tolist())
#     nan_idx = list(nan_idx)

#     kept_idx = np.ones(stimOn_times.shape[0]).astype(bool)
#     kept_idx[nan_idx] = False
    
#     stimOn_times = stimOn_times[kept_idx]
#     firstMovement_times = firstMovement_times[kept_idx]
#     goCue_times = goCue_times[kept_idx]
#     response_times = response_times[kept_idx]
#     choice = choice[kept_idx]
#     feedbackType = feedbackType[kept_idx]
#     feedback_times = feedback_times[kept_idx]
#     stimOff_times = stimOff_times[kept_idx]
#     stimOff_diff = stimOff_diff[kept_idx]

# #     diff = firstMovement_times-stimOn_times
# #     diff_idx = np.logical_and(diff > 0.0, diff < 1.0)
# #     diff = diff[diff_idx]
# #     print('max: {}, min: {}'.format(diff.max(), diff.min()))
# #     feedback_times = feedback_times[diff_idx]
# #     firstMovement_times = firstMovement_times[diff_idx]
    
# #     diff2 = feedback_times - firstMovement_times
# #     diff2_idx = np.logical_and(diff2 > 0.0, diff2 < 1.0)
# #     diff2 = diff2[diff2_idx]
# #     print('max: {}, min: {}'.format(diff2.max(), diff2.min()))
    
# #     print(diff2.shape)
#     stimOff_diff = stimOff_diff[np.where(stimOff_diff<10)]
#     print(stimOff_diff.shape)

#     print('--------------------------------------')

In [ ]:
mtnn_eids = get_mtnn_eids()
print(list(mtnn_eids.keys()))

In [ ]:
# run Charles's behavioral model
# run_exp_prevAction(mtnn_eids)

In [ ]:
traj = get_traj(mtnn_eids)
#print(traj)

In [ ]:
# ts_directory = os.path.join('.', "new_timestamps")
# if not os.path.exists(ts_directory):
#     os.makedirs(ts_directory)
# for i in notebook.tqdm(range(len(traj))):
#     eid = traj[i]['session']['id']
#     get_new_timestamps(eid, 'left', ts_directory)

In [ ]:
feature_list = []
output_list = []
cluster_number_list = []
trial_number_list = []
session_list = []
session_count = {'mainenlab': 0, 'churchlandlab': 0, 
                 ('hoferlab', 'mrsicflogellab'): 0, 
                 'danlab': 0, 'angelakilab':0}
for i in notebook.tqdm(range(len(traj))):
    feature, output, cluster_numbers, trial_numbers = featurize(i, traj[i], one, session_count)
    feature_list.append(feature)
    output_list.append(output)
    cluster_number_list.append(cluster_numbers)
    session_list.append(traj[i])
    trial_number_list.append(trial_numbers)

In [ ]:
for i in range(len(feature_list)):
    print(session_list[i]['session']['id'])
    print(feature_list[i].shape)
    np.save('./original_data/{}_feature.npy'.format(session_list[i]['session']['id']), feature_list[i])

In [ ]:
for i in range(len(output_list)):
    print(session_list[i]['session']['id'])
    print(output_list[i].shape)
    np.save('./original_data/{}_output.npy'.format(session_list[i]['session']['id']), output_list[i])

In [ ]:
for i in range(len(cluster_number_list)):
    print(session_list[i]['session']['id'])
    print(cluster_number_list[i].shape)
    np.save('./original_data/{}_clusters.npy'.format(session_list[i]['session']['id']), cluster_number_list[i])

In [ ]:
for i in range(len(session_list)):
    print(session_list[i]['session']['id'])
    np.save('./original_data/{}_session_info.npy'.format(session_list[i]['session']['id']), session_list[i])

In [ ]:
for i in range(len(trial_number_list)):
    print(session_list[i]['session']['id'])
    print(trial_number_list[i].shape)
    np.save('./original_data/{}_trials.npy'.format(session_list[i]['session']['id']), trial_number_list[i])

In [ ]:
def select_responsive_neurons(feature, output, clusters, neuron_id_start=0, threshold=5.0):
    select = output.mean(1).max(1) >= threshold
    feature_subset = feature[select]
    for i in range(feature_subset.shape[0]):
        feature_subset[i,:,:,0] = neuron_id_start+i
    
    return feature_subset, output[select], clusters[select]

In [ ]:
total_n_neurons = 0
shape_list = []
output_subset_list = []
cluster_subset_list = []
for i in notebook.tqdm(range(len(traj))):
    feature_subset, output_subset, clusters_subset = select_responsive_neurons(feature_list[i], 
                                                                               output_list[i],
                                                                               cluster_number_list[i],
                                                                               neuron_id_start=total_n_neurons,
                                                                               threshold=5.0)
    total_n_neurons += feature_subset.shape[0]
    print('{}/{} removed'.format(feature_subset.shape[0],feature_list[i].shape[0]))
    print('{}/{} remaining'.format(feature_list[i].shape[0]-feature_subset.shape[0],feature_list[i].shape[0]))
    shape_list.append(feature_subset.shape)
    output_subset_list.append(output_subset)
    cluster_subset_list.append(clusters_subset)
    
    if i == 0:
        feature_concat = feature_subset.reshape((-1,)+feature_subset.shape[-2:])
    else:
        feature_concat = np.concatenate((feature_concat, feature_subset.reshape((-1,)+feature_subset.shape[-2:])))
print('feature_concat shape: {}'.format(feature_concat.shape))

In [ ]:
def preprocess_feature(feature_concat):
    # normalize truncated_feature: dlc features + xyz location
    # first normalize xyz
    x_max = feature_concat[:,:,xyz_offset].max()
    x_min = feature_concat[:,:,xyz_offset].min()
    y_max = feature_concat[:,:,xyz_offset+1].max()
    y_min = feature_concat[:,:,xyz_offset+1].min()
    z_max = feature_concat[:,:,xyz_offset+2].max()
    z_min = feature_concat[:,:,xyz_offset+2].min()

    feature_concat[:,:,xyz_offset] = 0.1 + 0.9*(feature_concat[:,:,xyz_offset] - x_min) / (x_max - x_min)
    feature_concat[:,:,xyz_offset+1] = 0.1 + 0.9*(feature_concat[:,:,xyz_offset+1] - y_min) / (y_max - y_min)
    feature_concat[:,:,xyz_offset+2] = 0.1 + 0.9*(feature_concat[:,:,xyz_offset+2] - z_min) / (z_max - z_min)

    # next normalize dlc features
    for i in range(stimulus_offset - paw_offset):
        idx = paw_offset+i

        feature_min = feature_concat[:,:,idx].min()
        feature_max = feature_concat[:,:,idx].max()

        feature_concat[:,:,idx] = -1 + 2*(feature_concat[:,:,idx] - feature_min) / (feature_max - feature_min)

    # next normalize wheel
    wheel_min = feature_concat[:,:,wheel_offset].min()
    wheel_max = feature_concat[:,:,wheel_offset].max()

    feature_concat[:,:,wheel_offset] = -1 + 2*(feature_concat[:,:,wheel_offset] - wheel_min) / (wheel_max - wheel_min)

    # next normalize max_ptp
    max_ptp_min = feature_concat[:,:,max_ptp_offset].min()
    max_ptp_max = feature_concat[:,:,max_ptp_offset].max()

    feature_concat[:,:,max_ptp_offset] = 0.1 + 0.9*(feature_concat[:,:,max_ptp_offset] - max_ptp_min) / (max_ptp_max - max_ptp_min)

    # next normalize wf_width
    wf_width_min = feature_concat[:,:,wf_width_offset].min()
    wf_width_max = feature_concat[:,:,wf_width_offset].max()

    feature_concat[:,:,wf_width_offset] = 0.1 + 0.9*(feature_concat[:,:,wf_width_offset] - wf_width_min) / (wf_width_max - wf_width_min)
    
    # noise
    noise = np.random.normal(size=feature_concat.shape[:-1])
    feature_concat[:,:,noise_offset] = noise
    
    return feature_concat

In [ ]:
preprocessed_feature = preprocess_feature(feature_concat)
print(preprocessed_feature.shape)

In [ ]:
preprocessed_feature_list = []
idx = 0
for sh in shape_list:
    n = sh[0]*sh[1]
    preprocessed_feature_list.append(preprocessed_feature[idx:idx+n].reshape(sh))
    idx += n

In [ ]:
train_shape_list = []
val_shape_list = []
test_shape_list = []

train_bool_list = []
val_bool_list = []
test_bool_list = []

train_trial_list = []
val_trial_list = []
test_trial_list = []

train_feature = []
val_feature = []
test_feature = []

train_output = []
val_output = []
test_output = []

for i in notebook.tqdm(range(len(traj))):
    print(session_list[i]['session']['id'])
    
    n_trials = preprocessed_feature_list[i].shape[1]
    n_test = int(n_trials*0.2)
    n_train = int((n_trials-n_test)*0.8)
    n_val = n_trials - n_train - n_test
    
    sh = shape_list[i]
    train_shape_list.append((sh[0],n_train,)+sh[-2:])
    val_shape_list.append((sh[0],n_val,)+sh[-2:])
    test_shape_list.append((sh[0],n_test,)+sh[-2:])
    
    test_idx = np.random.choice(np.arange(n_trials), size=n_test, replace=False)
    test_bool = np.zeros(n_trials).astype(bool)
    test_bool[test_idx] = True
    
    train_idx = np.random.choice(np.arange(n_trials)[~test_bool], size=n_train, replace=False)
    train_bool = np.zeros(n_trials).astype(bool)
    train_bool[train_idx] = True
    
    val_bool = np.zeros(n_trials).astype(bool)
    val_bool[~np.logical_or(test_bool, train_bool)] = True
    
    train_bool_list.append(train_bool)
    val_bool_list.append(val_bool)
    test_bool_list.append(test_bool)
    
    train_trial_list.append(trial_number_list[i][train_bool])
    val_trial_list.append(trial_number_list[i][val_bool])
    test_trial_list.append(trial_number_list[i][test_bool])
    
    train_feature.append(preprocessed_feature_list[i][:,train_bool].reshape((-1,)+sh[-2:]))
    val_feature.append(preprocessed_feature_list[i][:,val_bool].reshape((-1,)+sh[-2:]))
    test_feature.append(preprocessed_feature_list[i][:,test_bool].reshape((-1,)+sh[-2:]))
    
    train_output.append(output_subset_list[i][:,train_bool].reshape(-1, sh[-2]))
    val_output.append(output_subset_list[i][:,val_bool].reshape(-1, sh[-2]))
    test_output.append(output_subset_list[i][:,test_bool].reshape(-1, sh[-2]))

In [ ]:
# np.save('mtnn_data/train/shape.npy', np.asarray(train_shape_list))
# np.save('mtnn_data/validation/shape.npy', np.asarray(val_shape_list))
# np.save('mtnn_data/test/shape.npy', np.asarray(test_shape_list))

# np.save('mtnn_data/train/bool.npy', np.asarray(train_bool_list, dtype=object))
# np.save('mtnn_data/validation/bool.npy', np.asarray(val_bool_list, dtype=object))
# np.save('mtnn_data/test/bool.npy', np.asarray(test_bool_list, dtype=object))

# np.save('mtnn_data/train/trials.npy', np.asarray(train_trial_list, dtype=object))
# np.save('mtnn_data/validation/trials.npy', np.asarray(val_trial_list, dtype=object))
# np.save('mtnn_data/test/trials.npy', np.asarray(test_trial_list, dtype=object))

# np.save('mtnn_data/train/feature.npy', np.concatenate(train_feature))
# np.save('mtnn_data/validation/feature.npy', np.concatenate(val_feature))
# np.save('mtnn_data/test/feature.npy', np.concatenate(test_feature))

# np.save('mtnn_data/train/output.npy', np.concatenate(train_output))
# np.save('mtnn_data/validation/output.npy', np.concatenate(val_output))
# np.save('mtnn_data/test/output.npy', np.concatenate(test_output))

np.save('mtnn_data/session_info.npy', np.asarray(session_list))
np.save('mtnn_data/clusters.npy', np.asarray(cluster_subset_list, dtype=object))